In [1]:
## Import the pip dependencies
import pandas as pd
from datetime import datetime

import sys
!{sys.executable} -m pip install azure-ai-evaluation

In [2]:
dataset = pd.read_csv("./slm_perf_output.csv")

sample = dataset.iloc[0]
print(sample)

Prompt      [{'role': 'system', 'content': 'You are a help...
Actual      {'tool_uses': [{'recipient_name': 'functions.s...
Expected    {'tool_uses': [{'recipient_name': 'functions.s...
Match                                                      No
Name: 0, dtype: object


In [3]:
import yaml
from azure.ai.evaluation import BleuScoreEvaluator, GleuScoreEvaluator, MeteorScoreEvaluator, RougeScoreEvaluator, RougeType

bleu = BleuScoreEvaluator()
glue = GleuScoreEvaluator()
meteor = MeteorScoreEvaluator(alpha = 0.9, beta = 3.0, gamma = 0.5)
rouge = RougeScoreEvaluator(rouge_type=RougeType.ROUGE_L)


In [4]:
with open('./llama-fc_config.yaml') as f:
    d = yaml.load(f, Loader=yaml.FullLoader)
    
AZURE_SUBSCRIPTION_ID = d['config']['AZURE_SUBSCRIPTION_ID']
AZURE_RESOURCE_GROUP = d['config']['AZURE_RESOURCE_GROUP']
AZURE_WORKSPACE = d['config']['AZURE_WORKSPACE']
AZURE_DATA_NAME = d['config']['AZURE_SFT_DATA_NAME']    
DATA_DIR = d['config']['SFT_DATA_DIR']
CLOUD_DIR = d['config']['CLOUD_DIR']
HF_MODEL_NAME_OR_PATH = d['config']['HF_MODEL_NAME_OR_PATH']


In [5]:
df = pd.read_csv("./slm_perf_output.csv")
df.rename(columns = {"Actual" : "ground_truth", "Expected" : "response"}, inplace = True)
df.to_json("./eval_baseline_data.jsonl", orient = "records", lines = True)

In [24]:
! az ad user show --id priyakedia@ --query objectId  

In [ ]:
# <mySubscriptionID>: Subscription ID of the Azure AI Studio hub's linked storage account (available in Azure AI hub resource view in Azure Portal).
# <myResourceGroupName>: Resource group of the Azure AI Studio hub's linked storage account.
# <user-id>: User object ID for role assignment (retrieve with "az ad user show" command).

! az role assignment create --role "Storage Blob Data Contributor" --scope /subscriptions/8cebb108-a4d5-402b-a0c4-f7556126277f/resourceGroups/openairesourcegroup_priya --assignee-principal-type User --assignee-object-id "8b6b15cc-9937-4208-8173-97ea0b9e1a13"

In [9]:
!az account show

In [12]:
from azure.ai.evaluation import evaluate

result = evaluate(
    data="./eval_baseline_data.jsonl",
    evaluators={
        "bleu": bleu,
        "gleu": glue,
        "meteor": meteor,
    },
    # Optionally provide your AI Studio project information to track your evaluation results in your Azure AI Studio project
    azure_ai_project={
        "subscription_id": AZURE_SUBSCRIPTION_ID,
        "resource_group_name": "openAIResourceGroup_priya",
        "project_name": "project-azureopenai-priya-westus",
    },
)

Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_bleu_bleu_asyncbleuscoreevaluator_a8924_xp_20250102_062320_143725
Prompt flow service has started...
Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_meteor_meteor_asyncmeteorscoreevaluator_d78kys2h_20250102_062320_151468
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_gleu_gleu_asyncgleuscoreevaluator_ve1ik3fl_20250102_062320_142846


EvaluationException: (InternalError) The workspace datastore secrets request failed with HTTP 400

In [10]:
result = bleu(response=sample["Actual"], ground_truth=sample["Expected"])

print(result)

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package punkt to /home/azureuser/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


{'bleu_score': 0.8557309221341731}


In [1]:
print(os.getcwd())

/mnt/batch/tasks/shared/LS_root/mounts/clusters/priyakedia-cpu/code/Users/priyakedia/fine-tune-llama-32b-fc


In [3]:
import os
import json  
  
# Function to read a JSONL file and return a list of dictionaries  
def read_jsonl(file_path):  
    data = []  
    with open(file_path, 'r', encoding='utf-8') as file:  
        for line in file:  
            # Parse the JSON object from each line and append it to the list  
            data.append(json.loads(line.strip()))  
    return data  
  
# Example usage
file_path = './eval_baseline_data.jsonl'  
jsonl_data = read_jsonl(file_path) 

print(jsonl_data)
  
# Print the data read from the JSONL file  
for i, record in enumerate(jsonl_data):
    if i>=2:
        break  
    print(record)  

/mnt/batch/tasks/shared/LS_root/mounts/clusters/priyakedia-cpu/code/Users/priyakedia/fine-tune-llama-32b-fc
[{'Prompt': '[{\'role\': \'system\', \'content\': \'You are a helpful assistant with access to the following functions. Use them if required -\\n{\\n    "name": "search_books",\\n    "description": "Search for books based on title or author",\\n    "parameters": {\\n        "type": "object",\\n        "properties": {\\n            "query": {\\n                "type": "string",\\n                "description": "The search query"\\n            },\\n            "author": {\\n                "type": "string",\\n                "description": "The author of the book"\\n            }\\n        }\\n    }\\n}\\n\\n{\\n    "name": "calculate_tip",\\n    "description": "Calculate the tip amount for a bill",\\n    "parameters": {\\n        "type": "object",\\n        "properties": {\\n            "bill_amount": {\\n                "type": "number",\\n                "description": "The tota

In [11]:
from azure.ai.evaluation import GleuScoreEvaluator

gleu = GleuScoreEvaluator()